In [26]:
import requests
import os
import yaml
import pandas as pd
import numpy as np
import re

- Import credentials

In [31]:
credentials = yaml.load(open(os.path.expanduser('~/.ssh/lending_club_api.yml')))

In [32]:
credentials

{'account_id': 71351615, 'authentication': 'lOYMBaOCpJorkX62/RCOZ1Vezn0='}

In [33]:
account_id = credentials['account_id']

In [36]:
account_id

71351615

- Make the api call for the portfolios owned
    - Find the API portfolio to put loans into

In [45]:
portfolio_r = requests.get("https://api.lendingclub.com/api/investor/v1/accounts/{}/portfolios".format(account_id),
                 headers={'Authorization': credentials['authentication'],'Content-Type': 'application/json',
                          'Accept': 'application/json'}, params = {'showAll':'true'})

In [47]:
portfolio_id = 0
for portfolio in portfolio_r.json()['myPortfolios']:
    if portfolio['portfolioName']=='api':
        portfolio_id = portfolio['portfolioId']
    

- Make the API call to see the loans listed

In [49]:

r = requests.get('https://api.lendingclub.com/api/investor/v1/loans/listing',
                 headers={'Authorization': credentials['authentication'],'Content-Type': 'application/json',
                          'Accept': 'application/json'}, params = {'showAll':'true'})

In [50]:
r

<Response [200]>

In [51]:
json_response =  r.json()

In [52]:
json_response.keys()

[u'loans', u'asOfDate']

In [53]:
test_loan = json_response['loans'][50]

In [54]:
test_loan['purpose']

u'debt_consolidation'

- Convert to DF

In [55]:
loans_df = pd.DataFrame(json_response['loans'])

In [56]:
loans_df.head()

,accNowDelinq,accOpenPast24Mths,acceptD,addrState,addrZip,allUtil,annualInc,annualIncJoint,applicationType,avgCurBal,...,totCollAmt,totCurBal,totHiCredLim,totalAcc,totalBalExMort,totalBalIl,totalBcLimit,totalCuTl,totalIlHighCreditLimit,totalRevHiLim
0,0,7,2017-09-19T07:36:37.000-07:00,CO,800xx,64.1,45000.0,85000.0,JOINT,7039,...,4300,70390,88947,34,70390,66141.0,10800,4,72447,16500
1,0,9,2017-08-31T12:05:58.000-07:00,WI,530xx,23.0,27887.0,NaN,INDIVIDUAL,1035,...,0,14498,34289,17,14498,8974.0,18050,2,10539,23750
2,0,1,2017-09-21T18:28:57.000-07:00,TX,761xx,63.9,38000.0,79000.0,JOINT,2911,...,0,26198,40980,13,26198,14788.0,17000,0,18780,22200
3,0,10,2017-09-15T09:21:16.000-07:00,WY,980xx,58.7,167000.0,NaN,INDIVIDUAL,5382,...,0,64581,110029,27,64581,64514.0,10200,10,70729,39300
4,1,3,2017-08-28T13:59:18.000-07:00,AZ,852xx,51.6,100000.0,NaN,INDIVIDUAL,30102,...,0,240814,299519,20,48508,44191.0,4900,2,86958,17100


In [57]:
loans_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 629 entries, 0 to 628
Data columns (total 103 columns):
accNowDelinq                  int64
accOpenPast24Mths             int64
acceptD                       object
addrState                     object
addrZip                       object
allUtil                       float64
annualInc                     float64
annualIncJoint                float64
applicationType               object
avgCurBal                     int64
bcOpenToBuy                   float64
bcUtil                        float64
chargeoffWithin12Mths         int64
collections12MthsExMed        int64
creditPullD                   object
delinq2Yrs                    int64
delinqAmnt                    float64
desc                          object
dti                           float64
dtiJoint                      float64
earliestCrLine                object
empLength                     float64
empTitle                      object
expD                          object
exp

In [59]:
loans_df.id.head()

0    119361021
1    117290003
2    119702021
3    119201589
4    117874429
Name: id, dtype: int64

#### Convert EmpLength to years

In [60]:
loans_df.empLength = loans_df.empLength.apply(lambda x: x/60 if x >0 else 0)

#### Create month to int 1-12

In [61]:
loans_df['month'] = loans_df.acceptD.apply(lambda x: pd.to_datetime(x).month)

#### Crea cols term_36 and term_60

In [62]:
loans_df['term_36'] = [1 if i ==36 else 0 for i in loans_df.term]

In [63]:
loans_df['term_60'] = [1 if i ==60 else 0 for i in loans_df.term]

#### Create grade columns
- a:g

In [64]:
loans_df['grade_a'] = [1 if i =='A' else 0 for i in loans_df.grade]
loans_df['grade_b']= [1 if i =='B' else 0 for i in loans_df.grade]
loans_df['grade_c']= [1 if i =='C' else 0 for i in loans_df.grade]
loans_df['grade_d']= [1 if i =='D' else 0 for i in loans_df.grade]
loans_df['grade_e']= [1 if i =='E' else 0 for i in loans_df.grade]
loans_df['grade_f']= [1 if i =='F' else 0 for i in loans_df.grade]
loans_df['grade_g']= [1 if i =='G' else 0 for i in loans_df.grade]

### Create home ownership columns
-'home_ownership_ANY',
 'home_ownership_MORTGAGE',
 'home_ownership_NONE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',

In [65]:
loans_df['home_ownership_any'] = [1 if i =='ANY' else 0 for i in loans_df.homeOwnership]
loans_df['home_ownership_mortgage'] = [1 if i =='MORTGAGE' else 0 for i in loans_df.homeOwnership]
loans_df['home_ownership_none'] = [1 if i =='NONE' else 0 for i in loans_df.homeOwnership]
loans_df['home_ownership_other'] = [1 if i =='OTHER' else 0 for i in loans_df.homeOwnership]
loans_df['home_ownership_own'] = [1 if i =='OWN' else 0 for i in loans_df.homeOwnership]
loans_df['home_ownership_rent'] = [1 if i =='RENT' else 0 for i in loans_df.homeOwnership]

## Create purpose columns
-'purpose_credit_card',
 'purpose_debt_consolidation',
 'purpose_educational',
 'purpose_home_improvement',
 'purpose_house',
 'purpose_major_purchase',
 'purpose_medical',
 'purpose_moving',
 'purpose_other',
 'purpose_renewable_energy',
 'purpose_small_business',
 'purpose_vacation',
 'purpose_wedding',

In [67]:
loans_df['purpose_credit_card'] = [1 if i =='credit_card' else 0 for i in loans_df.purpose]
loans_df['purpose_debt_consolidation'] = [1 if i =='debt_consolidation' else 0 for i in loans_df.purpose]
loans_df['purpose_educational'] = [1 if i =='educational' else 0 for i in loans_df.purpose]
loans_df['purpose_home_improvement'] = [1 if i =='home_improvement' else 0 for i in loans_df.purpose]
loans_df['purpose_house'] = [1 if i =='house' else 0 for i in loans_df.purpose]
loans_df['purpose_major_purchase'] = [1 if i =='major_purchase' else 0 for i in loans_df.purpose]
loans_df['purpose_medical'] = [1 if i =='medical' else 0 for i in loans_df.purpose]
loans_df['purpose_moving'] = [1 if i =='moving' else 0 for i in loans_df.purpose]
loans_df['purpose_other'] = [1 if i =='other' else 0 for i in loans_df.purpose]
loans_df['purpose_renewable_energy'] = [1 if i =='renewable_energy' else 0 for i in loans_df.purpose]
loans_df['purpose_small_business'] = [1 if i =='small_business' else 0 for i in loans_df.purpose]
loans_df['purpose_vacation'] = [1 if i =='vacation' else 0 for i in loans_df.purpose]
loans_df['purpose_wedding'] = [1 if i =='wedding' else 0 for i in loans_df.purpose]
loans_df['purpose_car'] = [1 if i =='car' else 0 for i in loans_df.purpose]




### Create the states columns
'addr_state_AK',
 'addr_state_AL',
 'addr_state_AR',
 'addr_state_AZ',
 'addr_state_CA',
 'addr_state_CO',
 'addr_state_CT',
 'addr_state_DC',
 'addr_state_DE',
 'addr_state_FL',
 'addr_state_GA',
 'addr_state_HI',
 'addr_state_IA',
 'addr_state_ID',
 'addr_state_IL',
 'addr_state_IN',
 'addr_state_KS',
 'addr_state_KY',
 'addr_state_LA',
 'addr_state_MA',
 'addr_state_MD',
 'addr_state_ME',
 'addr_state_MI',
 'addr_state_MN',
 'addr_state_MO',
 'addr_state_MS',
 'addr_state_MT',
 'addr_state_NC',
 'addr_state_ND',
 'addr_state_NE',
 'addr_state_NH',
 'addr_state_NJ',
 'addr_state_NM',
 'addr_state_NV',
 'addr_state_NY',
 'addr_state_OH',
 'addr_state_OK',
 'addr_state_OR',
 'addr_state_PA',
 'addr_state_RI',
 'addr_state_SC',
 'addr_state_SD',
 'addr_state_TN',
 'addr_state_TX',
 'addr_state_UT',
 'addr_state_VA',
 'addr_state_VT',
 'addr_state_WA',
 'addr_state_WI',
 'addr_state_WV',
 'addr_state_WY'

In [68]:
loans_df['addr_state_AK'] = [1 if i =='AK' else 0 for i in loans_df.addrState]
loans_df['addr_state_AL'] = [1 if i =='AL' else 0 for i in loans_df.addrState]
loans_df['addr_state_AR'] = [1 if i =='AR' else 0 for i in loans_df.addrState]
loans_df['addr_state_AZ'] = [1 if i =='AZ' else 0 for i in loans_df.addrState]
loans_df['addr_state_CA'] = [1 if i =='CA' else 0 for i in loans_df.addrState]
loans_df['addr_state_CO'] = [1 if i =='CO' else 0 for i in loans_df.addrState]
loans_df['addr_state_CT'] = [1 if i =='CT' else 0 for i in loans_df.addrState]
loans_df['addr_state_DC'] = [1 if i =='DC' else 0 for i in loans_df.addrState]
loans_df['addr_state_DE'] = [1 if i =='DE' else 0 for i in loans_df.addrState]
loans_df['addr_state_FL'] = [1 if i =='FL' else 0 for i in loans_df.addrState]
loans_df['addr_state_GA'] = [1 if i =='GA' else 0 for i in loans_df.addrState]
loans_df['addr_state_HI'] = [1 if i =='HI' else 0 for i in loans_df.addrState]
loans_df['addr_state_IA'] = [1 if i =='IA' else 0 for i in loans_df.addrState]
loans_df['addr_state_ID'] = [1 if i =='ID' else 0 for i in loans_df.addrState]
loans_df['addr_state_IL'] = [1 if i =='IL' else 0 for i in loans_df.addrState]
loans_df['addr_state_IN'] = [1 if i =='IN' else 0 for i in loans_df.addrState]
loans_df['addr_state_KS'] = [1 if i =='KS' else 0 for i in loans_df.addrState]
loans_df['addr_state_KY'] = [1 if i =='KY' else 0 for i in loans_df.addrState]
loans_df['addr_state_LA'] = [1 if i =='LA' else 0 for i in loans_df.addrState]
loans_df['addr_state_MA'] = [1 if i =='MA' else 0 for i in loans_df.addrState]
loans_df['addr_state_MD'] = [1 if i =='MD' else 0 for i in loans_df.addrState]
loans_df['addr_state_ME'] = [1 if i =='ME' else 0 for i in loans_df.addrState]
loans_df['addr_state_MI'] = [1 if i =='MI' else 0 for i in loans_df.addrState]
loans_df['addr_state_MN'] = [1 if i =='MN' else 0 for i in loans_df.addrState]
loans_df['addr_state_MO'] = [1 if i =='MO' else 0 for i in loans_df.addrState]
loans_df['addr_state_MS'] = [1 if i =='MS' else 0 for i in loans_df.addrState]
loans_df['addr_state_MT'] = [1 if i =='MT' else 0 for i in loans_df.addrState]
loans_df['addr_state_NC'] = [1 if i =='NC' else 0 for i in loans_df.addrState]
loans_df['addr_state_ND'] = [1 if i =='ND' else 0 for i in loans_df.addrState]
loans_df['addr_state_NE'] = [1 if i =='NE' else 0 for i in loans_df.addrState]
loans_df['addr_state_NH'] = [1 if i =='NH' else 0 for i in loans_df.addrState]
loans_df['addr_state_NJ'] = [1 if i =='NJ' else 0 for i in loans_df.addrState]
loans_df['addr_state_NM'] = [1 if i =='NM' else 0 for i in loans_df.addrState]
loans_df['addr_state_NV'] = [1 if i =='NV' else 0 for i in loans_df.addrState]
loans_df['addr_state_NY'] = [1 if i =='NY' else 0 for i in loans_df.addrState]
loans_df['addr_state_OH'] = [1 if i =='OH' else 0 for i in loans_df.addrState]
loans_df['addr_state_OK'] = [1 if i =='OK' else 0 for i in loans_df.addrState]
loans_df['addr_state_OR'] = [1 if i =='OR' else 0 for i in loans_df.addrState]
loans_df['addr_state_PA'] = [1 if i =='PA' else 0 for i in loans_df.addrState]
loans_df['addr_state_RI'] = [1 if i =='RI' else 0 for i in loans_df.addrState]
loans_df['addr_state_SC'] = [1 if i =='SC' else 0 for i in loans_df.addrState]
loans_df['addr_state_SD'] = [1 if i =='SD' else 0 for i in loans_df.addrState]
loans_df['addr_state_TN'] = [1 if i =='TN' else 0 for i in loans_df.addrState]
loans_df['addr_state_TX'] = [1 if i =='TX' else 0 for i in loans_df.addrState]
loans_df['addr_state_UT'] = [1 if i =='UT' else 0 for i in loans_df.addrState]
loans_df['addr_state_VA'] = [1 if i =='VA' else 0 for i in loans_df.addrState]
loans_df['addr_state_VT'] = [1 if i =='VT' else 0 for i in loans_df.addrState]
loans_df['addr_state_WA'] = [1 if i =='WA' else 0 for i in loans_df.addrState]
loans_df['addr_state_WI'] = [1 if i =='WI' else 0 for i in loans_df.addrState]
loans_df['addr_state_WV'] = [1 if i =='WV' else 0 for i in loans_df.addrState]
loans_df['addr_state_WY'] = [1 if i =='WY' else 0 for i in loans_df.addrState]



#### Create the zip code column

In [69]:
int(loans_df.addrZip[0][:3])

800

- Drop null zip codes

In [70]:
loans_df.addrZip = loans_df.addrZip.apply( lambda x: float(x[:3]))

In [82]:
loans_df.columns

Index([     u'accNowDelinq', u'accOpenPast24Mths',           u'acceptD',
               u'addrState',           u'addrZip',           u'allUtil',
               u'annualInc',    u'annualIncJoint',   u'applicationType',
               u'avgCurBal',
       ...
           u'addr_state_SD',     u'addr_state_TN',     u'addr_state_TX',
           u'addr_state_UT',     u'addr_state_VA',     u'addr_state_VT',
           u'addr_state_WA',     u'addr_state_WI',     u'addr_state_WV',
           u'addr_state_WY'],
      dtype='object', length=184)

In [71]:
# Columns to match the data model
# empLength is in months - convert to years
api_cols =['loanAmount',
           'fundedAmount',
           'intRate',
           'installment',
           'empLength',
           'annualInc',
           'addrZip',
          'dti',
           'delinq2Yrs',
           'inqLast6Mths',
       'mthsSinceLastDelinq',
           'mthsSinceLastRecord',
           'openAcc',
           'pubRec',
           'revolBal',
           'revolUtil',
           'totalAcc',
      'collections12MthsExMed',
           'mthsSinceLastMajorDerog',
           'accNowDelinq',
           'totCollAmt',
           'totCurBal',
           'openAcc6m',
      'openIl6m',
        'openIl12m',
           'openIl24m',
           'totalBalIl',
           'iLUtil',
           'maxBalBc',
           'allUtil',
        'totalRevHiLim',
           'inqLast12m',
           'accOpenPast24Mths',
      'avgCurBal',
           'bcUtil',
           'chargeoffWithin12Mths',
           'delinqAmnt',
           'moSinOldIlAcct',
           'moSinOldRevTlOp',
           'moSinRcntRevTlOp',
     'moSinRcntTl',
           'mortAcc',
           'mthsSinceRecentBc',
           'mthsSinceRecentBcDlq',
           'mthsSinceRecentInq',
           'mthsSinceRecentRevolDelinq',
      'numAcctsEver120Ppd',
           'numActvBcTl',
           'numActvRevTl',
           'numBcSats',
           'numBcTl',
           'numIlTl',
           'numOpRevTl',
           'numRevAccts',
      'numRevTlBalGt0',
           'numSats',
           'numTl120dpd2m',
           'numTl30dpd',
           'numTl90gDpd24m',
           'numTlOpPast12m',
           'pctTlNvrDlq',
      'percentBcGt75',
           'pubRecBankruptcies',
           'taxLiens',
           'totHiCredLim',
           'totalBalExMort',
           'totalBcLimit',
           'totalIlHighCreditLimit',
      'month',
           'term_36',
           'term_60',
           'grade_a',
           'grade_b',
           'grade_c',
           'grade_d',
           'grade_e',
           'grade_f',
           'grade_g',
      'home_ownership_any','home_ownership_mortgage','home_ownership_none','home_ownership_other','home_ownership_own',
      'home_ownership_rent',
           'purpose_car','purpose_credit_card','purpose_debt_consolidation','purpose_educational','purpose_home_improvement',
      'purpose_house','purpose_major_purchase','purpose_medical','purpose_moving','purpose_other','purpose_renewable_energy',
      'purpose_small_business','purpose_vacation','purpose_wedding',
           'addr_state_AK','addr_state_AL','addr_state_AR',
      'addr_state_AZ','addr_state_CA','addr_state_CO','addr_state_CT','addr_state_DC','addr_state_DE','addr_state_FL',
      'addr_state_GA','addr_state_HI','addr_state_IA','addr_state_ID','addr_state_IL','addr_state_IN','addr_state_KS',
      'addr_state_KY','addr_state_LA','addr_state_MA','addr_state_MD','addr_state_ME','addr_state_MI','addr_state_MN',
      'addr_state_MO','addr_state_MS','addr_state_MT','addr_state_NC','addr_state_ND','addr_state_NE','addr_state_NH',
      'addr_state_NJ','addr_state_NM','addr_state_NV','addr_state_NY','addr_state_OH','addr_state_OK','addr_state_OR',
      'addr_state_PA','addr_state_RI','addr_state_SC','addr_state_SD','addr_state_TN','addr_state_TX','addr_state_UT',
      'addr_state_VA','addr_state_VT','addr_state_WA','addr_state_WI','addr_state_WV','addr_state_WY']



In [72]:
len(api_cols)

149

### Create the final data frame

In [74]:
final_api_df = loans_df[api_cols]

# Data Model
#### This is what the model was trained on
```'loan_amnt',
 'funded_amnt',
 'int_rate',
 'installment',
 'emp_length',
 'annual_inc',
 'zip_code',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 'open_il_6m',
 'open_il_12m',
 'open_il_24m',
 'total_bal_il',
 'il_util',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_last_12m',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_util',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'mths_since_recent_bc',
 'mths_since_recent_bc_dlq',
 'mths_since_recent_inq',
 'mths_since_recent_revol_delinq',
 'num_accts_ever_120_pd',
 'num_actv_bc_tl',
 'num_actv_rev_tl',
 'num_bc_sats',
 'num_bc_tl',
 'num_il_tl',
 'num_op_rev_tl',
 'num_rev_accts',
 'num_rev_tl_bal_gt_0',
 'num_sats',
 'num_tl_120dpd_2m',
 'num_tl_30dpd',
 'num_tl_90g_dpd_24m',
 'num_tl_op_past_12m',
 'pct_tl_nvr_dlq',
 'percent_bc_gt_75',
 'pub_rec_bankruptcies',
 'tax_liens',
 'tot_hi_cred_lim',
 'total_bal_ex_mort',
 'total_bc_limit',
 'total_il_high_credit_limit',
 'month',
 'term_ 36 months',
 'term_ 60 months',
 'grade_A',
 'grade_B',
 'grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'home_ownership_ANY',
 'home_ownership_MORTGAGE',
 'home_ownership_NONE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'purpose_car',
 'purpose_credit_card',
 'purpose_debt_consolidation',
 'purpose_educational',
 'purpose_home_improvement',
 'purpose_house',
 'purpose_major_purchase',
 'purpose_medical',
 'purpose_moving',
 'purpose_other',
 'purpose_renewable_energy',
 'purpose_small_business',
 'purpose_vacation',
 'purpose_wedding',
 'addr_state_AK',
 'addr_state_AL',
 'addr_state_AR',
 'addr_state_AZ',
 'addr_state_CA',
 'addr_state_CO',
 'addr_state_CT',
 'addr_state_DC',
 'addr_state_DE',
 'addr_state_FL',
 'addr_state_GA',
 'addr_state_HI',
 'addr_state_IA',
 'addr_state_ID',
 'addr_state_IL',
 'addr_state_IN',
 'addr_state_KS',
 'addr_state_KY',
 'addr_state_LA',
 'addr_state_MA',
 'addr_state_MD',
 'addr_state_ME',
 'addr_state_MI',
 'addr_state_MN',
 'addr_state_MO',
 'addr_state_MS',
 'addr_state_MT',
 'addr_state_NC',
 'addr_state_ND',
 'addr_state_NE',
 'addr_state_NH',
 'addr_state_NJ',
 'addr_state_NM',
 'addr_state_NV',
 'addr_state_NY',
 'addr_state_OH',
 'addr_state_OK',
 'addr_state_OR',
 'addr_state_PA',
 'addr_state_RI',
 'addr_state_SC',
 'addr_state_SD',
 'addr_state_TN',
 'addr_state_TX',
 'addr_state_UT',
 'addr_state_VA',
 'addr_state_VT',
 'addr_state_WA',
 'addr_state_WI',
 'addr_state_WV',
 'addr_state_WY'] ```

In [75]:
total_x_cols = ['loan_amnt',
 'funded_amnt',
 'int_rate',
 'installment',
 'emp_length',
 'annual_inc',
 'zip_code',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 'open_il_6m',
 'open_il_12m',
 'open_il_24m',
 'total_bal_il',
 'il_util',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_last_12m',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_util',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'mths_since_recent_bc',
 'mths_since_recent_bc_dlq',
 'mths_since_recent_inq',
 'mths_since_recent_revol_delinq',
 'num_accts_ever_120_pd',
 'num_actv_bc_tl',
 'num_actv_rev_tl',
 'num_bc_sats',
 'num_bc_tl',
 'num_il_tl',
 'num_op_rev_tl',
 'num_rev_accts',
 'num_rev_tl_bal_gt_0',
 'num_sats',
 'num_tl_120dpd_2m',
 'num_tl_30dpd',
 'num_tl_90g_dpd_24m',
 'num_tl_op_past_12m',
 'pct_tl_nvr_dlq',
 'percent_bc_gt_75',
 'pub_rec_bankruptcies',
 'tax_liens',
 'tot_hi_cred_lim',
 'total_bal_ex_mort',
 'total_bc_limit',
 'total_il_high_credit_limit',
 'month',
 'term_ 36 months',
 'term_ 60 months',
 'grade_A',
 'grade_B',
 'grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'home_ownership_ANY',
 'home_ownership_MORTGAGE',
 'home_ownership_NONE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'purpose_car',
 'purpose_credit_card',
 'purpose_debt_consolidation',
 'purpose_educational',
 'purpose_home_improvement',
 'purpose_house',
 'purpose_major_purchase',
 'purpose_medical',
 'purpose_moving',
 'purpose_other',
 'purpose_renewable_energy',
 'purpose_small_business',
 'purpose_vacation',
 'purpose_wedding',
 'addr_state_AK',
 'addr_state_AL',
 'addr_state_AR',
 'addr_state_AZ',
 'addr_state_CA',
 'addr_state_CO',
 'addr_state_CT',
 'addr_state_DC',
 'addr_state_DE',
 'addr_state_FL',
 'addr_state_GA',
 'addr_state_HI',
 'addr_state_IA',
 'addr_state_ID',
 'addr_state_IL',
 'addr_state_IN',
 'addr_state_KS',
 'addr_state_KY',
 'addr_state_LA',
 'addr_state_MA',
 'addr_state_MD',
 'addr_state_ME',
 'addr_state_MI',
 'addr_state_MN',
 'addr_state_MO',
 'addr_state_MS',
 'addr_state_MT',
 'addr_state_NC',
 'addr_state_ND',
 'addr_state_NE',
 'addr_state_NH',
 'addr_state_NJ',
 'addr_state_NM',
 'addr_state_NV',
 'addr_state_NY',
 'addr_state_OH',
 'addr_state_OK',
 'addr_state_OR',
 'addr_state_PA',
 'addr_state_RI',
 'addr_state_SC',
 'addr_state_SD',
 'addr_state_TN',
 'addr_state_TX',
 'addr_state_UT',
 'addr_state_VA',
 'addr_state_VT',
 'addr_state_WA',
 'addr_state_WI',
 'addr_state_WV',
 'addr_state_WY']

In [76]:
y_col = ['repaid']

In [77]:
len(total_x_cols)

149

In [78]:
len(api_cols)

149

- Same number of columns and in the correct order 


### Impute zeros for NaNs

In [79]:
final_api_df.fillna(0,inplace=True)

/usr/local/lib/python2.7/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [80]:
final_api_df.iloc[:,:20] .head()

,loanAmount,fundedAmount,intRate,installment,empLength,annualInc,addrZip,dti,delinq2Yrs,inqLast6Mths,mthsSinceLastDelinq,mthsSinceLastRecord,openAcc,pubRec,revolBal,revolUtil,totalAcc,collections12MthsExMed,mthsSinceLastMajorDerog,accNowDelinq
0,5000.0,4775.0,20.00,185.82,0.0,45000.0,800.0,42.56,3,1,6.0,0.0,11,0,4249.0,12.5,34,0,11.0,0
1,8500.0,700.0,26.30,343.84,0.0,27887.0,530.0,32.29,0,1,0.0,0.0,15,0,5524.0,23.0,17,0,0.0,0
2,12000.0,11875.0,9.44,384.06,0.4,38000.0,761.0,18.92,0,0,0.0,0.0,10,0,11410.0,51.4,13,0,0.0,0
3,28000.0,26425.0,30.75,918.84,0.6,167000.0,980.0,12.07,0,2,0.0,110.0,13,1,67.0,0.2,27,0,0.0,0
4,30000.0,1975.0,12.62,1005.34,2.0,100000.0,852.0,24.29,2,0,1.0,0.0,8,0,4317.0,25.2,20,0,56.0,1


In [81]:
final_api_df.iloc[:,20:40] .head()

,totCollAmt,totCurBal,openAcc6m,openIl6m,openIl12m,openIl24m,totalBalIl,iLUtil,maxBalBc,allUtil,totalRevHiLim,inqLast12m,accOpenPast24Mths,avgCurBal,bcUtil,chargeoffWithin12Mths,delinqAmnt,moSinOldIlAcct,moSinOldRevTlOp,moSinRcntRevTlOp
0,4300,70390,2,2,1,4,66141.0,84.8,2588.0,64.1,16500,5,7,7039,12.1,1,0.0,114.0,132,6
1,0,14498,4,1,1,1,8974.0,85.0,2765.0,23.0,23750,3,9,1035,30.0,0,0.0,161.0,44,3
2,0,26198,0,1,0,1,14788.0,78.7,3457.0,63.9,22200,0,1,2911,52.6,0,0.0,18.0,135,52
3,0,64581,4,4,2,6,64514.0,91.2,0.0,58.7,39300,5,10,5382,0.0,0,0.0,160.0,99,6
4,0,240814,0,3,0,1,44191.0,60.4,3897.0,51.6,17100,1,3,30102,88.1,0,25.0,182.0,162,71


# Create orders
```
URL: https://api.lendingclub.com/api/investor/<version>/accounts/<investor id>/orders


{
	"aid":11111,
	"orders":[
	{
		"loanId":22222,
		"requestedAmount":55.0,
		"portfolioId":44444
	},
	{
		"loanId": 33333,
		"requestedAmount":25,
		"portfolioId":55555
	},
	{
		"loanId": 44444,
		"requestedAmount":25,
	}]
}```

In [167]:
loan_ids = loans_df.id